In [ ]:
ROOT_DIR =r'D:\John\harry Uni\detection\train'

In [ ]:
import torch

# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    # Get the number of available GPUs
    gpu_count = torch.cuda.device_count()
    print(f"Number of available GPUs: {gpu_count}")
    
    # Get the name of the GPU device(s)
    for i in range(gpu_count):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("CUDA is not available. Using CPU...")
!pip install ultralytics


In [ ]:
conda remove pytorch torchvision torchaudio cudatoolkit
conda clean --all

In [ ]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    # Create a tensor on the GPU
    device = torch.device("cuda")
    tensor = torch.rand(3, 3).to(device)
    print("CUDA is available. Tensor is placed on GPU.")
else:
    print("CUDA is not available. Make sure CUDA drivers are installed correctly.")


In [ ]:
import torch
print(torch.cuda.is_available())

In [ ]:
import os
from ultralytics import YOLO

# Load a YOLOv8 model from a configuration file
model = YOLO("yolov8n.yaml")  # build a new model from scratch

# Define the path to your data configuration file
data_config_path = r"D:\John\harry Uni\detection\train\google_colab_config.yaml"

# Define the output directory where you want to save the training data
output_dir = r"D:\John\harry Uni\detection\output"

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Move the model to GPU if CUDA is available
if torch.cuda.is_available():
    model.cuda()

# Train the model using the specified data configuration and number of epochs
results = model.train(data=data_config_path, epochs=1, project=output_dir)  # train the model and save outputs

print(f"Training results are saved in: {output_dir}")


In [ ]:
!scp -r /content/runs '/content/gdrive/My Drive/ComputerVisionDeveloper/TrainYolov8CustomDataset'

In [1]:
from ultralytics import YOLO
import cv2
import math 
# start webcam
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

# model
checkpoint_path = r"D:\John\harry Uni\detection\output\train\weights\best.pt"
model = YOLO(checkpoint_path)

# object classes
classNames = ["Bottle"
              ]


while True:
    success, img = cap.read()
    results = model(img, stream=True)

    # coordinates
    for r in results:
        boxes = r.boxes

        for box in boxes:
            # bounding box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values

            # put box in cam
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), 3)

            # confidence
            confidence = math.ceil((box.conf[0]*100))/100
            print("Confidence --->",confidence)

            # class name
            cls = int(box.cls[0])
            print("Class name -->", classNames[cls])

            # object details
            org = [x1, y1]
            font = cv2.FONT_HERSHEY_SIMPLEX
            fontScale = 1
            color = (255, 0, 0)
            thickness = 2

            cv2.putText(img, classNames[cls], org, font, fontScale, color, thickness)

    cv2.imshow('Webcam', img)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 480x640 (no detections), 199.5ms
Speed: 0.0ms preprocess, 199.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 159.8ms
Speed: 4.1ms preprocess, 159.8ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 169.2ms
Speed: 4.1ms preprocess, 169.2ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 168.2ms
Speed: 4.8ms preprocess, 168.2ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 164.8ms
Speed: 2.0ms preprocess, 164.8ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 177.9ms
Speed: 0.0ms preprocess, 177.9ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 162.1ms
Speed: 2.8ms preprocess, 162.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 158.4ms
Speed: 3.2ms prepr

In [1]:
!pip install keras
!pip install tensorflow

In [4]:
from ultralytics import YOLO
import cv2
import math
import tensorflow as tf
import numpy as np

# Load the Keras model
keras_model_path = r"D:\John\harry Uni\25x2x0.keras"
keras_model = tf.keras.models.load_model(keras_model_path)

# Function to preprocess images
def preprocess_image(image):
    # Resize to 100x100 and convert to grayscale
    resized_image = cv2.resize(image, (100, 100))
    grayscale_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    # Normalize
    normalized_image = grayscale_image / 255.0
    # Add channel dimension
    processed_image = np.expand_dims(normalized_image, axis=-1)
    return processed_image

# Start webcam
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

# YOLO model
checkpoint_path = r"D:\John\harry Uni\detection\output\train\weights\best.pt"
model = YOLO(checkpoint_path)

# Object classes
classNames = ["Bottle"]

while True:
    success, img = cap.read()
    results = model(img, stream=True)

    # Coordinates
    for r in results:
        boxes = r.boxes

        for box in boxes:
            # Bounding box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)  # Convert to int values

            # Put box in cam
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), 3)

            # Extract the detected bottle region
            bottle_region = img[y1:y2, x1:x2]

            # Preprocess the image
            preprocessed_image = preprocess_image(bottle_region)

            # Predict using the Keras model
            prediction = keras_model.predict(np.array([preprocessed_image]))

            # Confidence
            confidence = math.ceil((box.conf[0] * 100)) / 100
            print("Confidence --->", confidence)

            # Classify whether the bottle is beer or plastic
            class_index = np.argmax(prediction)
            class_name = "Beer" if class_index == 0 else "Plastic"
            class_confidence = np.max(prediction)

            print("Class name -->", class_name)
            print("Confidence -->", class_confidence)

            # Display classification result
            org = [x1, y1]
            font = cv2.FONT_HERSHEY_SIMPLEX
            fontScale = 1
            color = (255, 0, 0)
            thickness = 2
            cv2.putText(img, f"{class_name} ({class_confidence:.2f})", org, font, fontScale, color, thickness)

    cv2.imshow('Webcam', img)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 480x640 (no detections), 176.1ms
Speed: 8.5ms preprocess, 176.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 160.0ms
Speed: 2.0ms preprocess, 160.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 163.7ms
Speed: 2.2ms preprocess, 163.7ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 158.1ms
Speed: 0.0ms preprocess, 158.1ms inference, 15.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 168.1ms
Speed: 3.1ms preprocess, 168.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 161.6ms
Speed: 2.0ms preprocess, 161.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 165.1ms
Speed: 0.0ms preprocess, 165.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 164.3ms
Speed: 0.0ms prep

In [7]:
import os

# Path to the training images and labels
train_path = r"D:\John\harry Uni\detection\OIDv4_ToolKit\OID\test\train\Bottle"

# List all image files
image_files = [f for f in os.listdir(train_path) if f.endswith(('.jpg', '.jpeg', '.png'))]

# Check for corresponding label files and validate content
missing_labels = []
corrupt_labels = []

for image_file in image_files:
    label_file = os.path.splitext(image_file)[0] + '.txt'
    label_file_path = os.path.join(train_path, label_file)
    
    if not os.path.exists(label_file_path):
        missing_labels.append(image_file)
        continue
    
    # Validate label file content
    with open(label_file_path, 'r') as file:
        lines = file.readlines()
    
    if not lines:
        corrupt_labels.append(label_file_path)
        continue
    
    for line in lines:
        parts = line.strip().split()
        if len(parts) != 5:
            corrupt_labels.append(label_file_path)
            break
        
        try:
            class_id = int(parts[0])
            x_center = float(parts[1])
            y_center = float(parts[2])
            width = float(parts[3])
            height = float(parts[4])
            if not (0 <= x_center <= 1 and 0 <= y_center <= 1 and 0 <= width <= 1 and 0 <= height <= 1):
                corrupt_labels.append(label_file_path)
                break
        except ValueError:
            corrupt_labels.append(label_file_path)
            break

if missing_labels:
    print(f"Missing label files for {len(missing_labels)} images: {missing_labels}")
else:
    print("All images have corresponding label files.")

if corrupt_labels:
    print(f"Corrupt label files: {corrupt_labels}")
else:
    print("All label files are correctly formatted.")


All images have corresponding label files.
All label files are correctly formatted.


In [ ]:
import os

# Define the mapping from class names to indices
class_mapping = {
    'Bottle': 0
}

# Path to the training images and labels
train_path = r"D:\John\harry Uni\detection\train\Bottle\Label\new"

# List all label files
label_files = [f for f in os.listdir(train_path) if f.endswith('.txt')]

def is_valid_coordinate(value):
    """Check if the coordinate is a valid normalized float between 0 and 1."""
    try:
        value = float(value)
        return 0 <= value <= 1
    except ValueError:
        return False

for label_file in label_files:
    label_file_path = os.path.join(train_path, label_file)
    
    # Read the label file
    with open(label_file_path, 'r') as file:
        lines = file.readlines()
    
    fixed_lines = []
    for line in lines:
        parts = line.strip().split()
        
        # Check if the first part is a class name that needs conversion
        if parts[0] in class_mapping:
            parts[0] = str(class_mapping[parts[0]])
        
        # Check if we have exactly 5 parts and all coordinates are valid
        if len(parts) == 5 and all(is_valid_coordinate(part) for part in parts[1:]):
            fixed_lines.append(" ".join(parts))
        else:
            print(f"Invalid or corrupt label found in {label_file_path}: {line.strip()}")
            # Attempt to fix common issues:
            if len(parts) == 5:
                try:
                    parts[0] = str(int(parts[0]))  # Convert class id to integer
                    parts[1] = str(float(parts[1])) if is_valid_coordinate(parts[1]) else "0.5"
                    parts[2] = str(float(parts[2])) if is_valid_coordinate(parts[2]) else "0.5"
                    parts[3] = str(float(parts[3])) if is_valid_coordinate(parts[3]) else "0.1"
                    parts[4] = str(float(parts[4])) if is_valid_coordinate(parts[4]) else "0.1"
                    fixed_lines.append(" ".join(parts))
                    print(f"Fixed corrupt label in {label_file_path}: {' '.join(parts)}")
                except ValueError:
                    print(f"Could not fix label in {label_file_path}: {line.strip()}")
            else:
                print(f"Skipped invalid label in {label_file_path}: {line.strip()}")
    
    # Write the fixed labels back to the file
    with open(label_file_path, 'w') as file:
        file.write("\n".join(fixed_lines))

print("Label files have been checked and fixed where possible.")


In [ ]:
import os
from PIL import Image

# Directory containing text files
text_files_dir = r"D:\John\harry Uni\detection\OIDv4_ToolKit\OID\test\train\Bottle\Label"

# Output directory for YOLO format labels
yolo_labels_dir = r"D:\John\harry Uni\detection\OIDv4_ToolKit\OID\test\train\Bottle"

# Create the output directory if it doesn't exist
os.makedirs(yolo_labels_dir, exist_ok=True)

# Function to normalize coordinates and adjust them according to image size
def convert_to_yolo_format(x_min, y_min, x_max, y_max, img_width, img_height):
    x_center = (x_min + x_max) / (2 * img_width)
    y_center = (y_min + y_max) / (2 * img_height)
    width = (x_max - x_min) / img_width
    height = (y_max - y_min) / img_height
    return x_center, y_center, width, height

# Process each text file
for filename in os.listdir(text_files_dir):
    if filename.endswith(".txt"):
        file_path = os.path.join(text_files_dir, filename)
        img_path = os.path.join(r"D:\John\harry Uni\detection\OIDv4_ToolKit\OID\test\train\Bottle", os.path.splitext(filename)[0] + ".jpg")  # Corresponding image file
        
        # Read image dimensions
        with Image.open(img_path) as img:
            img_width, img_height = img.size
        
        # Read lines from the text file
        with open(file_path, 'r') as file:
            lines = file.readlines()
        
        # Create a new YOLO format label file
        yolo_label_path = os.path.join(yolo_labels_dir, filename)
        
        with open(yolo_label_path, 'w') as yolo_label_file:
            for line in lines:
                # Split the line into components (class, x_min, y_min, x_max, y_max)
                components = line.strip().split()
                
                # Ensure that there are 5 components
                if len(components) != 5:
                    print(f"Invalid line in {filename}: {line.strip()}")
                    continue
                
                # Convert bounding box coordinates to YOLO format
                class_id = components[0]
                x_min, y_min, x_max, y_max = map(float, components[1:])
                x_center, y_center, width, height = convert_to_yolo_format(x_min, y_min, x_max, y_max, img_width, img_height)
                
                # Write the converted line to the YOLO label file
                yolo_label_file.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

print("Conversion to YOLO format completed.")


In [ ]:
import os

directory = r"D:\John\harry Uni\detection\OIDv4_ToolKit\OID\test\train\Bottle"

# Iterate through all files in the directory
for filename in os.listdir(directory):
    # Check if the file is a text file
    if filename.endswith(".txt"):
        filepath = os.path.join(directory, filename)
        # Open the file for reading
        with open(filepath, "r") as file:
            content = file.read()
        # Replace "bottle" with "0" in the content
        modified_content = content.replace("Bottle", "0")
        # Open the file for writing and save the modified content
        with open(filepath, "w") as file:
            file.write(modified_content)

print("Text files modified successfully!")


In [ ]:
import os
import cv2
from tqdm import tqdm
from ultralytics import YOLO

# Directory containing images and annotations
images_dir = r"D:\John\harry Uni\detection\OIDv4_ToolKit\OID\test\train\Bottle"
annotations_dir = r"D:\John\harry Uni\detection\OIDv4_ToolKit\OID\test\train\Bottle"

# Initialize variables for evaluation metrics
total_precision = 0.0
total_recall = 0.0
total_f1_score = 0.0
total_images = 0

# Load the YOLO model
model = YOLO(r"D:\John\harry Uni\detection\output\train\weights\best.pt")

# Iterate through images in the directory
for filename in tqdm(os.listdir(images_dir)):
    if filename.endswith(".jpg"):
        # Load image
        image_path = os.path.join(images_dir, filename)
        image = cv2.imread(image_path)

        # Perform object detection
        results = model(image)

        # Parse ground truth annotations
        annotation_path = os.path.join(annotations_dir, filename.replace(".jpg", ".txt"))
        ground_truth_boxes = parse_annotation_file(annotation_path)

        # Calculate evaluation metrics
        # (replace this with your actual evaluation code)
        precision = 0.0
        recall = 0.0
        f1_score = 0.0

        # Update total evaluation metrics
        total_precision += precision
        total_recall += recall
        total_f1_score += f1_score
        total_images += 1

# Calculate average evaluation metrics
average_precision = total_precision / total_images
average_recall = total_recall / total_images
average_f1_score = total_f1_score / total_images

print("Average Precision:", average_precision)
print("Average Recall:", average_recall)
print("Average F1 Score:", average_f1_score)


In [ ]:
import os
import cv2
from ultralytics import YOLO

# Directory containing images and annotations
images_dir = r"D:\John\harry Uni\detection\OIDv4_ToolKit\OID\test\train\Bottle"
annotations_dir = r"D:\John\harry Uni\detection\OIDv4_ToolKit\OID\test\train\Bottle"

# Load the YOLO model
model = YOLO(r"D:\John\harry Uni\detection\output\train\weights\best.pt")

# Iterate through images in the directory
for filename in os.listdir(images_dir):
    if filename.endswith(".jpg"):
        # Load image
        image_path = os.path.join(images_dir, filename)
        image = cv2.imread(image_path)

        # Perform object detection
        results = model(image)

        # Parse ground truth annotations
        annotation_path = os.path.join(annotations_dir, filename.replace(".jpg", ".txt"))
        with open(annotation_path, "r") as file:
            annotations = file.readlines()
        
        # Draw ground truth bounding boxes
        for annotation in annotations:
            class_id, x_center, y_center, width, height = map(float, annotation.strip().split())
            x1 = int((x_center - width / 2) * image.shape[1])
            y1 = int((y_center - height / 2) * image.shape[0])
            x2 = int((x_center + width / 2) * image.shape[1])
            y2 = int((y_center + height / 2) * image.shape[0])
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
        # Draw detected bounding boxes
        for box in results.xyxy[0]:
            x1, y1, x2, y2 = map(int, box[:4])
            cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)
        
        # Show image with bounding boxes
        cv2.imshow("Image", image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
